In [ ]:
%run my_import
#from my_import import *
import numpy as np

#uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
uri = "com-exp.local:5555"
#uri = "mio-thales.local:5555"
#uri = "mirror-ecat-srv.local:5555"
#uri = "192.168.99.10:5555"

io = set_uri(uri)
io.debug = False
scan_ids = []


# all 
robot_ids = [11,12,13,14,15,16,17]
#robot_ids = [4]

home = { iD: 0 for iD in robot_ids }
#home[14] = -1.45

homing = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "HOMING", "name": "homing", "board_id": 1, "homing_par": {"x": [0,3]}}}

xs = list(range(9))
xs = np.multiply(xs,2)
smooth = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": 1,
                             "smooth_par": {"x": xs,
                                            "y": [0,-0.5, 0, 0.5, 0, -1.3, 0, 1.3, 0]}
                                            #"y": [0,-0.2, 0, 0.2, 0, -0.2, 0, 0.2, 0]}
                            }
         } 

xss = list(range(5))
xss = np.multiply(xss,2)
pitch = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "pitch_smooth", "board_id": -1,
                               "smooth_par": {"x": xss,
                                            "y": [0,-0.7, 0, 1.0, 0]}
                            }
         } 


park_x6 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "park_smooth", "board_id": -1,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, -0.7]}
                               }
    }
park_x4 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "park_smooth", "board_id": -1,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, -1.8]}
                               }
    }
park_x2 = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "park_smooth", "board_id": -1,
                                "smooth_par": {"x": [0,3],
                                               "y": [0, 1.3]}
                               }
    }


add_homing = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing"]}}

add_smooth = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["smooth"]}}

add_park = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["park_smooth"]}}

add_pitch = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["pitch_smooth"]}}



In [ ]:
reply = io.doit(master_cmd_get_slave_descr)
slave_descr = yaml.safe_load(reply['msg'])
scan_ids = slave_descr.keys()
scan_ids

# Define trajectories

In [ ]:
# define trajectories
# reset trj names ???
[g for g in io.doit4ids(robot_ids,homing)]
[g for g in io.doit4ids(robot_ids,smooth)]
# overwrite "smooth" for pitch(s) 
#[g for g in io.doit4ids([4],smooth_x4)]
#
[g for g in io.doit4ids([12],park_x2)]
[g for g in io.doit4ids([14],park_x4)]
[g for g in io.doit4ids([16],park_x6)]
# pitch 
[g for g in io.doit4ids([12,14,16],pitch)]


In [ ]:
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_set_home)]
for iD in robot_ids :
    io.doit(ctrl_cmd_set_home.set_value(home[iD]).set_bid(iD))

# Position test

In [ ]:
# start motors in POSITION
[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x3B).set_gains(u'POSITION',220,35,0,0,0))]
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',2000,40,1,0.4,0.01))]


In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', iD)

In [ ]:
reply = io.doit(add_homing)

In [ ]:
reply = io.doit(add_smooth)

In [ ]:
reply = io.doit(add_pitch)

In [ ]:
reply = io.doit(add_park)

In [ ]:
for i in range(3):
    reply = io.doit(add_smooth)
    reply = io.doit(add_pitch)
    reply = io.doit(add_homing)
    reply = io.doit(add_park)
    reply = io.doit(add_homing)
    print ("CICLO ", i)
    #time.sleep(1)

In [ ]:
reply = io.doit(add_park)

In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', iD)

In [ ]:
[g for g in io.doit4ids(robot_ids,ctrl_cmd_stop)]

In [ ]:
io.debug=0
gains = {}
gains['type'] = "POSITION"
gains['pos_kp'] = 200
gains['pos_kd'] = 10
gains['tor_kp'] = 0
gains['tor_kd'] = 0
gains['tor_ki'] = 0
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x3B).set_gains(u'POSITION',200,15,0,0,0))]
[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',300,10,1,0.75,0.005))]
#[g for g in io.doit4ids(robot_ids,ctrl_cmd_start().set_value(0x3B).set_gains(*gains))]

In [ ]:
rd_sdo=['motor_pos','motor_vel','link_pos','fault','master_cmd_status_copy']
ret=read_sdo(rd_sdo,robot_ids)
pp(ret)
rd_sdo=['gain_0','gain_1','gain_2','gain_','gain_4']
#ret=read_sdo(rd_sdo,robot_ids)
#pp(ret)

In [ ]:
io.debug=0
for iD in robot_ids:
    reply = io.doit(SdoCmd(rd_sdo=['gain_0','gain_1','gain_2','gain_','gain_4'],wr_sdo={}).set_bid(iD))
    yaml_msg = yaml.safe_load(reply['msg'])
    print(iD,yaml_msg)

# Impedance

In [ ]:
# start motors in IMPEDANCE
# 1st yaws stiff 200 or 70 damp 10
[g for g in io.doit4ids([12],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',700,10,1,0.85,0.007))]
[g for g in io.doit4ids([14],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',500,10,1,0.85,0.007))]
[g for g in io.doit4ids([16],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',100,10,1,0.85,0.007))]
# other yaws + last pitch stiff 100 or 30 damp 10
[g for g in io.doit4ids([11,13,15],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',200,10,1,0.85,0.007))]
[g for g in io.doit4ids([17],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',100,10,1,0.85,0.01))]
# 1st 2nd pitches stiff 700 or 250 damp 10
#[g for g in io.doit4ids([12,13,22,23,32,33],ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',700,10,1,0.75,0.007))]
# 3rd leg
#[g for g in io.doit4ids(_3rdL,ctrl_cmd_stop)]
#[g for g in io.doit4ids(_3rdL,ctrl_cmd_start().set_value(0x4D).set_gains(u'IMPEDANCE',0,2,1,0.85,0.005))]


In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', iD)

In [ ]:
reply = io.doit(add_homing)

In [ ]:
reply = io.doit(add_park)

In [ ]:
reply = io.doit(add_pitch)

In [ ]:
for i in range(2):
    reply = io.doit(add_smooth)
    rreply = io.doit(add_pitch)
    reply = io.doit(add_homing)
    print ("CICLO ", i)

In [ ]:
reply = io.doit(add_park)

In [ ]:
for iD in robot_ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', iD)

In [ ]:
[g for g in io.doit4ids(robot_ids,ctrl_cmd_stop)]

In [ ]:
np.multiply(list(range(9)),10)

In [ ]:
home[4]

In [ ]:
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(11))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(17))

In [ ]:
sdo_dict